In [ ]:
import xarray as xr
import numpy as np
import hvplot.xarray
import hvplot.pandas
from glob import glob
from tqdm import tqdm
from holoviews.util.transform import lon_lat_to_easting_northing
import cmasher as cmr
from datashader import transfer_functions as tf, reductions as rd
import holoviews as hv
from holoviews import opts
import geoviews as gv
import pandas as pd
import geopandas as gpd
import panel as pn
import matplotlib.pyplot as plt

In [ ]:
fnames = glob(f"../data/external/swot/*")
fnames.sort()
fnames = np.array(fnames)

In [ ]:
ds = xr.open_dataset(fnames[-2], group="pixel_cloud")
ds["latitude"].attrs["long_name"] = "latitude"
ds["longitude"].attrs["long_name"] = "longitude"
ds["latitude"].attrs["units"] = "°"
ds["longitude"].attrs["units"] = "°"
ds.load()


In [ ]:
image = xr.open_dataset("../data/external/landsat/after.nc").rename({"lon": "longitude", "lat": "latitude"})
image["longitude"].attrs = ds.longitude.attrs
image["latitude"].attrs = ds.latitude.attrs

scl = image["SCL"].load()

In [ ]:
rgb = xr.concat([image[b] for b in ["B4", "B3", "B2"]], "bands").rename("image").assign_coords(bands=[0,1,2]).load()

# Clip outliers (e.g., 2nd and 98th percentiles)
vmin, vmax = rgb.quantile([0.02, 0.98])
rgb_clipped = rgb.clip(vmin, vmax)

# Scale to 0-1 range (linear scaling)
rgb_scaled = (rgb_clipped - vmin) / (vmax - vmin)

# Gamma correction (optional, for better visual contrast)
gamma = 0.8
rgb_gamma_corrected = rgb_scaled ** (1/gamma)

rgb_gamma_corrected = rgb_gamma_corrected.transpose("latitude","longitude","bands")

In [ ]:
height = 300
width = 500

plot_kwargs = {
    "swot": {
        "title": "",
        "height": height,
        "width": width,
        "x": "longitude",
        "y": "latitude",
        "geo": True,
        "rasterize": True,
        "cmap": [
            [0, 160, 0],     # land - green
            [255, 230, 90],  # land near water - dark yellow
            [100, 200, 255], # water near land - very bright blue
            [0, 0, 255],     # open water - blue
            [14, 27, 102],   # dark water - dark blue
            [192, 192, 192], # low coherence water near land - grey
            [255, 150, 255], # open low coherence water - very bright pink
        ],
        "clim": (1, 7),
        "rescale_discrete_levels": True,
        "x_sampling": 0.0002,
        "y_sampling": 0.0002,
        "aggregator": "mean",
        "colorbar": False
    },
    "scl": {
        "title": "",
        "height": height,
        "width": width,
        "x": "longitude",
        "y": "latitude",
        "geo": True,
        "rasterize": True,
        "cmap": [
            [0, 0, 0],       # No Data (Missing data) - black
            [255, 0, 0],     # Saturated or defective pixel - red
            [47, 47, 47],    # Topographic casted shadows - very dark grey
            [100, 50, 0],    # Cloud shadows - dark brown
            [0, 160, 0],     # Vegetation - green
            [255, 230, 90],  # Not-vegetated - dark yellow
            [0, 0, 255],     # Water (dark and bright) - blue
            [128, 128, 128], # Unclassified - dark grey
            [192, 192, 192], # Cloud medium probability - grey
            [255, 255, 255], # Cloud high probability - white
            [100, 200, 255], # Thin cirrus - very bright blue
            [255, 150, 255], # Snow or ice - very bright pink
        ],
        "clim": (0, 11),
        "rescale_discrete_levels": True,
        "colorbar": False,
    },
    "landsat": {
        "title": "",
        "height": height,
        "width": width,
        "x": "longitude",
        "y": "latitude",
        "bands": "bands",
        "geo": True,
        "rasterize": True,
    },
    "water_depth": {
        "title": "",
        "height": height,
        "width": width,
        "x": "longitude",
        "y": "latitude",
        "color": "water_depth",
        "colorbar": False,
        "clim": (0,12),
        "rasterize": True,
        "geo": True,
        "x_sampling": 0.0002,
        "y_sampling": 0.0002,
        "aggregator": "mean",
    }
}

# Elevation

In [ ]:
elevation = xr.open_dataset("../data/external/landsat/elevation.nc").elevation.rename({"lon": "longitude", "lat": "latitude"})
elevation["longitude"].attrs = ds.longitude.attrs
elevation["latitude"].attrs = ds.latitude.attrs

ds["elevation"] = elevation.interp(longitude=ds.longitude, latitude=ds.latitude)
ds["scl"] = scl.sel(longitude=ds.longitude, latitude=ds.latitude, method="nearest")
height_above_geoid = (ds.height-ds.geoid)
ds["water_depth"] = (height_above_geoid-ds.elevation)

In [ ]:
height_above_geoid = (ds.height-ds.geoid)
ds["water_depth"] = (height_above_geoid-ds.elevation)

In [ ]:
ind = np.argwhere(((ds.scl==6)|(ds.classification==4)).values).ravel()
water = ds.isel(points=ind)

In [ ]:
(
    scl.hvplot.quadmesh(**plot_kwargs["scl"])+
    ds.classification.hvplot.points(**plot_kwargs["swot"])+
    rgb_gamma_corrected.hvplot.rgb(**plot_kwargs["landsat"])+
    water["water_depth"].hvplot.points(**plot_kwargs["water_depth"])
).cols(2)

In [ ]:
water.water_depth.hvplot.hist(bins=np.arange(-10,10,0.1))

In [ ]:
volume = (water.water_depth.where((water.water_depth>0)&(water.water_depth<10))*water.pixel_area).sum().values

In [ ]:
volume/1000

In [ ]:
cantareira = 982e6
amazonas = 209000

In [ ]:
(volume/amazonas)/3600

In [ ]:
volume/cantareira

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file("../data/external/shapefiles/RM_Porto_Alegre/RM_PortoAlegre_UDH.shp")
gdf = gdf.rename({"UDH_ATLAS": "udh"}, axis=1)
gdf["udh"] = gdf["udh"].astype("int")
gdf = gdf.set_index("udh")


In [ ]:
fname = "../data/external/shapefiles/RM_Porto_Alegre/atlasivs_dadosbrutos_Porto_Alegre.xlsx"
df = pd.read_excel(fname, sheet_name="UDH")
df = df[df.ano==2010].reset_index(drop=True).set_index("udh")
gdf = gdf.join(df.loc[gdf.index])

In [ ]:
clim = gdf.populacao.quantile([0.01, 0.99]).values
clim = tuple(clim.tolist())

In [ ]:
water_plot = water["water_depth"].hvplot.points(**plot_kwargs["water_depth"], hover=False)

In [ ]:
kw = dict(height=height, width=width, geo=True, line_width=0, colorbar=False)
(
    rgb_gamma_corrected.hvplot.rgb(**plot_kwargs["landsat"])+
    water["water_depth"].where(water["water_depth"]>0).hvplot.points(**plot_kwargs["water_depth"])+
    water_plot*gdf.hvplot(color="ivs_infraestrutura_urbana", cmap=cmr.flamingo_r, **kw)+
    water_plot*gdf.hvplot(color="populacao", cmap=cmr.toxic_r, clim=clim, **kw)
).cols(2)

In [ ]:
elevation.hvplot.hist()

In [ ]:
kw = dict(height=height, width=width, x="longitude", y="latitude", geo=True, rasterize=True, colorbar=False)

(
    rgb_gamma_corrected.hvplot.rgb(**plot_kwargs["landsat"])+
    elevation.where(elevation<6).hvplot(**kw)
).cols(1)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import fiona
import re
import html

url = "https://www.google.com/maps/d/u/0/viewer?mid=1ZzfSX_tPoDATywh1GptSdw5FDO9FVuU&l"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# (1) Find the script tag that contains the map data
scripts = soup.find_all('script')

# (2) Search for the variable containing the GeoJSON data
pattern = re.compile(r'var _pageData = (.*?);')
for script in soup.find_all('script'):
    match = pattern.search(script.text)
    if match:
        # (2) Decode and convert to a Python list
        page_data_string = html.unescape(match.group(1))
        page_data = json.loads(page_data_string)
        break
else:
    raise ValueError("Could not find _pageData in the HTML source.")

In [ ]:
# (3) Extract the GeoJSON data from page_data (this part is tricky and needs to be adjusted based on the actual structure)
geojson_data = None
for item in page_data:
    if isinstance(item, list):
        for subitem in item:
            if isinstance(subitem, list) and "Mancha de inundação" in subitem: 
                # This seems to be where the GeoJSON is stored
                # Adapt the indexing based on the structure
                geojson_data = subitem[2][0][4][0]
                break
        if geojson_data:
            break  # Exit the loop if we found the GeoJSON

In [ ]:
page_data_string = page_data_string.strip('"')
page_data_string = page_data_string.replace("\\u003d", "=")

page_data = json.loads(page_data_string)

In [ ]:
page_data_string.split('\\"')